In [1]:
import pandas as pd
import numpy
from sqlalchemy import create_engine
import psycopg2
import matplotlib.pyplot as plt
from itertools import cycle

In [2]:

dbname = 'xxxxx'
user='xxxxx'
host='xxxxxxxxx.rds.amazonaws.com'
port='5432'
password='xxxxxxxxx'


In [3]:
def runQuery(query):
    connect_text = "dbname='%s' user='%s' host=%s port=%s password='%s'" % (dbname, user, host, port, password) 
    con = psycopg2.connect(connect_text) 
    cur = con.cursor() 
    cur.execute(query) 
    con.commit() 
    con.close()

## SQL to manually created table first

In [4]:
sql = """
  create table us_road_network.trip_routes as
SELECT  a.trip_id,
        SUM(a.cost) AS agg_cost,
        ST_Union(b.geom) AS geom       -- ST_Collect(b.geom)
FROM    (
    SELECT  trip_id, 
            (pgr_dijkstra(
                'SELECT id::int, source::bigint, target::bigint, travel_cost::float as cost FROM us_road_network.roads_us',
                origin,
                destination,
                false
            )).*
    FROM    (SELECT * FROM (SELECT row_number() over(), * FROM us_road_network.routes_start_stop_points order by trip_id) t_r where row_number<10) t
) AS a
JOIN    us_road_network.roads_us AS b
  ON    a.edge = b.id
GROUP BY
        a.trip_id
ORDER BY
        a.trip_id
"""

## Linearly scale processinging to use the maximum number of CPUs available to db

In [5]:
queries = [] 

max_rows = 23065+25
start_row = 20

list_of_starts = []
for i in range(start_row, max_rows, 20):
    list_of_starts.append([str(i), str(i+20)])
    
print(len(list_of_starts))
list_of_starts[:3]


for l in list_of_starts:
    start = l[0]
    end = l[1]
    sql = f"""
      insert into us_road_network.trip_routes 
    SELECT  a.trip_id,
            SUM(a.cost) AS agg_cost,
            ST_Union(b.geom) AS geom       -- ST_Collect(b.geom)
    FROM    (
        SELECT  trip_id, 
                (pgr_dijkstra(
                    'SELECT id::int, source::bigint, target::bigint, travel_cost::float as cost FROM us_road_network.roads_us',
                    origin,
                    destination,
                    false
                )).*
        FROM    (SELECT * FROM 
        (SELECT row_number() over(), * FROM us_road_network.routes_start_stop_points order by trip_id) t_r where row_number>={start} and row_number<{end}
    ) t
    ) AS a
    JOIN    us_road_network.roads_us AS b
      ON    a.edge = b.id
    GROUP BY
            a.trip_id
    ORDER BY
            a.trip_id
    """
    queries.append(sql)

test
1154


In [6]:
%%time
import multiprocessing 
pool = multiprocessing.Pool(94)

CPU times: user 127 ms, sys: 350 ms, total: 477 ms
Wall time: 430 ms


In [7]:
%%time
for i in pool.imap_unordered(runQuery, queries): 
    continue

CPU times: user 2.11 s, sys: 1.08 s, total: 3.18 s
Wall time: 30min 35s
